In [1]:
from openff.toolkit import ForceField, Molecule, Topology
from openmm import LangevinMiddleIntegrator, unit
from openff.interchange import Interchange
import mdtraj

In [2]:
integrator = LangevinMiddleIntegrator(300, 1, 4)

In [3]:
print("Building ligand..")
ligand = Molecule.from_file("final.sdf", allow_undefined_stereo=True)

Building ligand..


In [4]:
ligand_intrcg = Interchange.from_smirnoff(force_field=ForceField("openff_unconstrained-2.0.0.offxml"), topology=[ligand],)

In [5]:
print("\nBuilding protein..")
protein = Topology.from_pdb("final.pdb") # apo protein + ions


Building protein..


In [6]:
protein_intrcg = Interchange.from_smirnoff(
    force_field=ForceField("ff14sb_off_impropers_0.0.3.offxml", "openff_unconstrained-2.0.0.offxml"),
    topology=protein,
)

In [7]:
%env INTERCHANGE_EXPERIMENTAL=1

env: INTERCHANGE_EXPERIMENTAL=1


In [8]:
print("\nBuilding complex..")
complex_intrcg = protein_intrcg + ligand_intrcg


Building complex..


/lila/home/takabak/mambaforge/envs/espfit/lib/python3.11/site-packages/openff/interchange/_experimental.py:35: UserWarning: Interchange object combination is experimental and likely to produce strange results. Any workflow using this method is not guaranteed to be suitable for production. Use with extreme caution and thoroughly validate results!
  return func(*args, **kwargs)


In [9]:
print("Minimizing..")
simulation = complex_intrcg.to_openmm_simulation(integrator)
start_pot_energy = simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole
simulation.minimizeEnergy(maxIterations=3)
end_pot_energy = simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

Minimizing..


In [10]:
print(f"{round(start_pot_energy, 2)} -> {round(end_pot_energy, 2)}")

1231.43 -> -3746.86


In [11]:
# check complex topology
res = [ r.name for r in simulation.topology.residues() ]
set(res)

{'ALA',
 'ARG',
 'ASN',
 'ASP',
 'CYS',
 'GLN',
 'GLU',
 'GLY',
 'HIS',
 'ILE',
 'LEU',
 'LYS',
 'MET',
 'NA',
 'PHE',
 'PRO',
 'SER',
 'THR',
 'TRP',
 'TYR',
 'UNK',
 'VAL'}

#### (Approach 1) Update protein positions and create new system

In [12]:
res = [ r.residue_name for r in protein.residues ]
set(res)

{'ALA',
 'ARG',
 'ASN',
 'ASP',
 'CYS',
 'GLN',
 'GLU',
 'GLY',
 'HIS',
 'ILE',
 'LEU',
 'LYS',
 'MET',
 'NA',
 'PHE',
 'PRO',
 'SER',
 'THR',
 'TRP',
 'TYR',
 'VAL'}

In [13]:
# initial positions
protein.get_positions()

Magnitude,[[0.6899000000000001 10.523499999999999 -0.9576] [0.6973999999999999 10.636400000000002 -0.8615999999999999] [0.6038999999999999 10.616500000000002 -0.7406999999999999] ... [7.344799999999998 1.4195999999999998 9.4656] [2.4410000000000003 5.6686000000000005 7.085899999999999] [2.8047 1.717 6.7174000000000005]]
Units,nanometer


In [14]:
# openmm topology
protein_topology = protein.to_openmm()
# number of protein atoms
n_protein_atoms = protein_topology.getNumAtoms()
# complex position (OpenMM Quantity -> need to convert this to OpenFF Quantity later)
complex_positions = simulation.context.getState(getPositions=True).getPositions()

In [15]:
# get minimized protein positions
import numpy as np
protein_positions = unit.Quantity(np.zeros([n_protein_atoms, 3]), unit=unit.nanometers)
protein_positions = complex_positions[:n_protein_atoms]

In [16]:
# we need to pass OpenFF Quantity
from openff.units.openmm import from_openmm
protein.set_positions(from_openmm(protein_positions))

In [17]:
# check updated positions
protein.get_positions()

Magnitude,[[0.6894777514478075 10.523000027908788 -0.9549346030380099] [0.6993661624424595 10.635254507118061 -0.8620176985340186] [0.6034363796335723 10.615794416485414 -0.7451342554810715] ... [7.344833803557084 1.419589102604206 9.46563146090298] [2.440951987889813 5.668701678320518 7.085871770429732] [2.804721252154227 1.7170858093330479 6.717395719630116]]
Units,nanometer


In [18]:
# create new system and compute potential energy
import copy
integrator_copy = copy.deepcopy(integrator)

protein_intrcg_2 = Interchange.from_smirnoff(
    force_field=ForceField("ff14sb_off_impropers_0.0.3.offxml", "openff_unconstrained-2.0.0.offxml"),
    topology=protein,
)
simulation_2 = protein_intrcg_2.to_openmm_simulation(integrator_copy)
energy = simulation_2.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

In [19]:
print(f"{round(energy, 2)}")

-3842.63


In [20]:
# check topology
res = [ r.name for r in simulation_2.topology.residues() ]
set(res)

{'ALA',
 'ARG',
 'ASN',
 'ASP',
 'CYS',
 'GLN',
 'GLU',
 'GLY',
 'HIS',
 'ILE',
 'LEU',
 'LYS',
 'MET',
 'NA',
 'PHE',
 'PRO',
 'SER',
 'THR',
 'TRP',
 'TYR',
 'VAL'}

#### (Approach 2) Turn off ligand FF parameters

In [21]:
res = [ r.name for r in simulation.topology.residues() ]
set(res)

{'ALA',
 'ARG',
 'ASN',
 'ASP',
 'CYS',
 'GLN',
 'GLU',
 'GLY',
 'HIS',
 'ILE',
 'LEU',
 'LYS',
 'MET',
 'NA',
 'PHE',
 'PRO',
 'SER',
 'THR',
 'TRP',
 'TYR',
 'UNK',
 'VAL'}

In [33]:
# ligand atom indices
#resnames = ['UNK', 'NA']
resnames = ['UNK']
ligand_atom_indices = []
for atom in simulation.topology.atoms():
    if atom.residue.name in resnames:
        ligand_atom_indices.append(atom.index)

In [34]:
ligand_atom_indices

[9372,
 9373,
 9374,
 9375,
 9376,
 9377,
 9378,
 9379,
 9380,
 9381,
 9382,
 9383,
 9384,
 9385,
 9386,
 9387,
 9388,
 9389,
 9390,
 9391,
 9392,
 9393,
 9394,
 9395,
 9396,
 9397,
 9398,
 9399,
 9400,
 9401,
 9402,
 9403,
 9404,
 9405,
 9406,
 9407,
 9408,
 9409,
 9410,
 9411,
 9412,
 9413,
 9414,
 9415,
 9416,
 9417,
 9418,
 9419,
 9420,
 9421,
 9422,
 9423,
 9424,
 9425,
 9426,
 9427,
 9428,
 9429,
 9430]

In [35]:
# get forces
forces = list(simulation.system.getForces())
forces

[<openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7fa348cf7ab0> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7fa348cf6be0> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7fa348cf4330> >,
 <openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7fa348cf4b10> >]

In [36]:
def check_ligand_forces(ligand_atom_indices):
    # check ligand forces
    for force in forces:
        name = force.__class__.__name__
    
        if "Bond" in name:
            for idx in range(force.getNumBonds()):
                id1, id2, length, k = force.getBondParameters(idx)
                sublist = [id1, id2]
                if all(x in ligand_atom_indices for x in sublist):
                    print(f'Bond: {id1}, {id2}, {length}, {k}')
    
        elif "Angle" in name:
            for idx in range(force.getNumAngles()):
                id1, id2, id3, angle, k = force.getAngleParameters(idx)
                sublist = [id1, id2, id3]
                if all(x in ligand_atom_indices for x in sublist):
                    print(f'Angle: {id1}, {id2}, {id3}, {angle}, {k}')
                    
        elif "Torsion" in name:
            for idx in range(force.getNumTorsions()):
                id1, id2, id3, id4, periodicity, phase, k = force.getTorsionParameters(idx)
                sublist = [id1, id2, id3, id4]
                if all(x in ligand_atom_indices for x in sublist):
                    print(f'Torsion: {id1}, {id2}, {id3}, {id4}, {periodicity}, {phase}, {k}')
        
        elif "Nonbonded" in name:
            for idx in range(force.getNumParticles()):
                if idx in ligand_atom_indices:
                    q, sigma, epsilon = force.getParticleParameters(idx)
                    print(f'Nonbonded: {idx}, {q}, {sigma}, {epsilon}')
            for idx in range(force.getNumExceptions()):
                if idx in ligand_atom_indices:
                    idx0, idx1, q, sigma, epsilon = force.getExceptionParameters(idx)
                    print(f'Nonbonded NumExceptions: {idx}, {idx0}, {idx1}, {q}, {sigma}, {epsilon}')

In [37]:
# check ligand forces before turning off FF parameters
check_ligand_forces(ligand_atom_indices)

Nonbonded: 9372, 0.07728306779661016 e, 0.337953176162662 nm, 0.45538911611061844 kJ/mol
Nonbonded: 9373, -0.5629169322033898 e, 0.3206876023663901 nm, 0.7016212989374017 kJ/mol
Nonbonded: 9374, -0.6131169322033898 e, 0.3039812205065809 nm, 0.8795023257036865 kJ/mol
Nonbonded: 9375, 0.6710830677966102 e, 0.34806468869450646 nm, 0.3635030558377792 kJ/mol
Nonbonded: 9376, -0.43481693220338985 e, 0.3206876023663901 nm, 0.7016212989374017 kJ/mol
Nonbonded: 9377, -0.5768169322033898 e, 0.2997159987248637 nm, 0.8764372596155737 kJ/mol
Nonbonded: 9378, -0.040616932203389834 e, 0.337953176162662 nm, 0.45538911611061844 kJ/mol
Nonbonded: 9379, -0.6630169322033899 e, 0.3206876023663901 nm, 0.7016212989374017 kJ/mol
Nonbonded: 9380, -0.6505169322033898 e, 0.3039812205065809 nm, 0.8795023257036865 kJ/mol
Nonbonded: 9381, -0.10141693220338983 e, 0.337953176162662 nm, 0.45538911611061844 kJ/mol
Nonbonded: 9382, -0.6560169322033899 e, 0.3206876023663901 nm, 0.7016212989374017 kJ/mol
Nonbonded: 9383, 

In [38]:
# check potential energy before turning off FF parameters
_energy = simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole
print(round(_energy, 2))

-3746.86


In [39]:
for force in forces:
    name = force.__class__.__name__

    if "Bond" in name:
        for idx in range(force.getNumBonds()):
            id1, id2, length, k = force.getBondParameters(idx)
            sublist = [id1, id2]
            if all(x in ligand_atom_indices for x in sublist):
                print(f'Bond: {sublist}')
                force.setBondParameters(idx, id1, id2, length, 0.0)
        force.updateParametersInContext(simulation.context)

    elif "Angle" in name:
        for idx in range(force.getNumAngles()):
            id1, id2, id3, angle, k = force.getAngleParameters(idx)
            sublist = [id1, id2, id3]      
            if all(x in ligand_atom_indices for x in sublist):
                print(f'Angle: {sublist}')
                force.setAngleParameters(idx, id1, id2, id3, angle, 0.0)
        force.updateParametersInContext(simulation.context)
                
    elif "Torsion" in name:
        for idx in range(force.getNumTorsions()):
            id1, id2, id3, id4, periodicity, phase, k = force.getTorsionParameters(idx)
            sublist = [id1, id2, id3, id4]      
            if all(x in ligand_atom_indices for x in sublist):
                print(f'Torsion: {sublist}')
                force.setTorsionParameters(idx, id1, id2, id3, id4, periodicity, phase, 0.0)
        force.updateParametersInContext(simulation.context)
    
    elif "Nonbonded" in name:
        for idx in range(force.getNumParticles()):
            if idx in ligand_atom_indices:
                print(f'Nonbonded: {idx}')
                q, sigma, epsilon = force.getParticleParameters(idx)
                
                # set partial charge q to 1e-10 to avoid following error:
                # OpenMMException: updateParametersInContext: The number of non-excluded exceptions has changed
                force.setParticleParameters(idx, q * 1e-10, 0, 0)
        for idx in range(force.getNumExceptions()):
            if idx in ligand_atom_indices:
                print(f'Nonbonded NumExceptions: {idx}')
                idx0, idx1, q, sigma, epsilon = force.getExceptionParameters(idx)
                force.setExceptionParameters(idx, idx0, idx1, q * 1e-10, 0, 0)
        force.updateParametersInContext(simulation.context)

Nonbonded: 9372
Nonbonded: 9373
Nonbonded: 9374
Nonbonded: 9375
Nonbonded: 9376
Nonbonded: 9377
Nonbonded: 9378
Nonbonded: 9379
Nonbonded: 9380
Nonbonded: 9381
Nonbonded: 9382
Nonbonded: 9383
Nonbonded: 9384
Nonbonded: 9385
Nonbonded: 9386
Nonbonded: 9387
Nonbonded: 9388
Nonbonded: 9389
Nonbonded: 9390
Nonbonded: 9391
Nonbonded: 9392
Nonbonded: 9393
Nonbonded: 9394
Nonbonded: 9395
Nonbonded: 9396
Nonbonded: 9397
Nonbonded: 9398
Nonbonded: 9399
Nonbonded: 9400
Nonbonded: 9401
Nonbonded: 9402
Nonbonded: 9403
Nonbonded: 9404
Nonbonded: 9405
Nonbonded: 9406
Nonbonded: 9407
Nonbonded: 9408
Nonbonded: 9409
Nonbonded: 9410
Nonbonded: 9411
Nonbonded: 9412
Nonbonded: 9413
Nonbonded: 9414
Nonbonded: 9415
Nonbonded: 9416
Nonbonded: 9417
Nonbonded: 9418
Nonbonded: 9419
Nonbonded: 9420
Nonbonded: 9421
Nonbonded: 9422
Nonbonded: 9423
Nonbonded: 9424
Nonbonded: 9425
Nonbonded: 9426
Nonbonded: 9427
Nonbonded: 9428
Nonbonded: 9429
Nonbonded: 9430
Nonbonded NumExceptions: 9372
Nonbonded NumExceptions: 9

In [40]:
# check ligand forces after turning off
check_ligand_forces(ligand_atom_indices)

Nonbonded: 9372, 7.728306779661017e-12 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9373, -5.6291693220338984e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9374, -6.131169322033899e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9375, 6.710830677966102e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9376, -4.3481693220338985e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9377, -5.7681693220338985e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9378, -4.0616932203389835e-12 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9379, -6.630169322033899e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9380, -6.505169322033898e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9381, -1.0141693220338983e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9382, -6.560169322033898e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9383, -1.0141693220338983e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9384, 1.0498306779661017e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9385, -5.671693220338984e-12 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9386, -9.041693220338982e-12 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9387, 1.677830677966102e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbo

In [41]:
# compute potential energy
energy_2 = simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

In [42]:
# energy difference between the two approaches
print(f"Approach 1: {round(energy, 2)}")
print(f"Approach 2: {round(energy_2, 2)}")

Approach 1: -3842.63
Approach 2: -3791.06
